In [10]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict


In [11]:
class BatsmanState(TypedDict):
    runs:int
    balls:int
    fours:int
    sixes:int
    sr:float
    bpb:float
    boundary_percent:float
    summary:str

In [19]:
def calculate_sr(state:BatsmanState):
    sr = state['runs']/state['balls']*100
    return {'sr': sr}

In [13]:
def calculate_bpb(state:BatsmanState)->BatsmanState:
    bpb = state['balls']/(state['fours']+state['sixes'])
    return {'bpb': bpb}

In [14]:
#calculates how much runs from boundary are scored runs/(four*4+six*6)*100
def calculate_boundary_percent(state:BatsmanState)->BatsmanState:
    boundary_percent = state['runs']/(state['fours']*4+state['sixes']*6)*100
    return {'boundary_percent': boundary_percent}

In [15]:
def summary(state:BatsmanState)->BatsmanState:
    summary = f"SR:{state['sr']}, BPB:{state['bpb']}, Boundary Percent:{state['boundary_percent']}"
    return {'summary': summary}

In [16]:
graph = StateGraph(BatsmanState)

graph.add_node('calculate_sr',calculate_sr)
graph.add_node('calculate_bpb',calculate_bpb)
graph.add_node('calculate_boundary_percent',calculate_boundary_percent)
graph.add_node('summary',summary)

#add edges
graph.add_edge(START,'calculate_sr')
graph.add_edge(START,'calculate_bpb')
graph.add_edge(START,'calculate_boundary_percent')

graph.add_edge('calculate_sr','summary')
graph.add_edge('calculate_bpb','summary')
graph.add_edge('calculate_boundary_percent','summary')

graph.add_edge('summary',END)

workflow = graph.compile()

In [20]:
initial_state = {
    'runs':100,
    'balls':37,
    'fours':6,
    'sixes':6
}
final_state = workflow.invoke(initial_state)
print(final_state)

{'runs': 100, 'balls': 37, 'fours': 6, 'sixes': 6, 'sr': 0.027027027027027025, 'bpb': 3.0833333333333335, 'boundary_percent': 166.66666666666669, 'summary': 'SR:0.027027027027027025, BPB:3.0833333333333335, Boundary Percent:166.66666666666669'}
